In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, element_at, slice, size, regexp_extract, transform, when, explode, \
monotonically_increasing_id, map_from_arrays, lit, udf,collect_list, row_number, ceil, map_keys, expr, from_json
from pyspark.sql.types import ShortType, ArrayType, LongType, StringType
from pyspark.sql import types as T

from IPython.display import display
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import mean_squared_error
from scipy.stats import zscore
import gzip
import os
import csv
from alive_progress import alive_bar
from collections import defaultdict
import pandas as pd
from os import listdir
from os.path import isfile, join

directory = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/'

def find_n_gram(direc, find):
    for filename in os.listdir(direc):
        f = os.path.join(direc, filename)
        # checking if it is a file
        if os.path.isfile(f):
            print(f)
            with gzip.open(f,'rt', encoding='utf-8') as input:
                str_arr_csv = input.readlines()
                for line in str_arr_csv:
                    token = line.split("\t")
                    if find == token[0]:
                        return line
                        
    return "line not found" 

def start_the_search(directory,search):
    result = find_n_gram(directory,search)
    
    if result == "line not found":
        print(result)
        exit()
    result = result.split("\t")
    lower_ngram = pd.Series(result[1:],dtype= "string", name = result[0])
    return lower_ngram
            

In [20]:
found = start_the_search(directory,"Geschlecht '")
print(found)

C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_aa.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ab.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ac.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ad.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ae.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_af.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ag.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ah.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ai.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_aj.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ak.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_al.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_am.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_an.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ao.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_ap.gz
C:/Users/bincl/BA-Thesis/Dataset/2gram/2_20000_nopos_aq.

In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, element_at, slice, size, regexp_extract, transform, when, explode, \
monotonically_increasing_id, map_from_arrays, lit, udf,collect_list, row_number, ceil, map_keys, expr, from_json
from pyspark.sql.types import ShortType, ArrayType, LongType, StringType
from pyspark.sql import types as T

import pandas as pd
from os import listdir
from os.path import isfile, join

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

years_Columns = list(range(1800,2000)) 
directory = 'C:/Users/bincl/BA-Thesis/Dataset/1gram/1_20000_nopos.gz'

#pathlist = [directory + f for f in listdir(directory) if isfile(join(directory, f))]

spark2 = SparkSession.builder.appName('3gramSQL').getOrCreate()

raw_input_df2 = spark2 \
                .read.csv(directory, sep='\n',quote="").withColumnRenamed('_c0', 'Input')

split_df2 = raw_input_df2 \
                .select(split('Input', '\t').alias('SplitInput')) \
                .select(element_at('SplitInput', 1).alias('Tokens'),
                        slice('SplitInput', 2,
size('SplitInput')).alias('Data')) \
                .select('Tokens', 'Data') \

df_2gram = split_df2.select('Tokens', transform('Data', lambda d:
split(d, ',')).alias('Data')) \
                .select('Tokens', transform('Data', lambda x:
x[0]).alias('Year'),
                        transform('Data', lambda x:
x[1]).cast(ArrayType(LongType())).alias('Occurrences')) \
                .select('Tokens', map_from_arrays('Year',
'Occurrences').alias('Data')) \
                .withColumn('Id', monotonically_increasing_id()) \
                .select(['Id', 'Tokens', 'Data'])


In [ ]:
def get_pd_df_from_string(suchstring, df):
    x = df_2gram
    x = x.where(x.Tokens == suchstring)
    id = x.first()['Id']
    print(id)
    return id 

: 

In [ ]:
matched_pandas_df = get_pd_df_from_string('Bis', df_2gram)

800
